In [1]:
import sys
import csv
from collections import OrderedDict
from typing import Tuple
import math
import random
import os
from copy import deepcopy
import numpy as np; np.random.seed(1)
import torch; torch.manual_seed(1)
from torch import nn, Tensor
import torch.nn.functional as F
from time import time
from tqdm import tqdm
from scipy import io
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR

torch.set_default_dtype(torch.float32)
import warnings
warnings.filterwarnings("ignore")
torch.set_printoptions(precision=5,sci_mode=False)

# Golbal Variables

In [2]:
DATASET = 'SUN' 
DATA_DIR = f'../../Datasets/{DATASET}'
DEVICE = 'cuda:0'

# Attribute Refinement Network

In [3]:
def exist(x):
    return x is not None

class Residual(nn.Module):
    def __init__(self,fn):
        super().__init__()
        self.fn=fn
    
    def forward(self,x):
        return self.fn(x)+x

class GatingUnit(nn.Module):
    def __init__(self,dim,len_sen):
        super().__init__()
        self.ln=nn.LayerNorm(dim)
        self.proj=nn.Conv1d(len_sen,len_sen,1)

        nn.init.zeros_(self.proj.weight)
        nn.init.ones_(self.proj.bias)
    
    def forward(self,x):
        res,gate=torch.chunk(x,2,-1)
        gate=self.ln(gate) 
        gate=self.proj(gate.unsqueeze(-1)) 
        gate = gate.squeeze(-1)
        return res*gate

class ARN(nn.Module):
    def __init__(self,num_tokens=None,len_sen=49,dim=512,d_ff=1024,num_layers=6):
        super().__init__()
        self.num_layers=num_layers
        self.embedding=nn.Embedding(num_tokens,dim) if exist(num_tokens) else nn.Identity()

        self.arn=nn.ModuleList([Residual(nn.Sequential(OrderedDict([
            ('ln1_%d'%i,nn.LayerNorm(dim)),
            ('fc1_%d'%i,nn.Linear(dim,d_ff*2)),
            ('gelu_%d'%i,nn.GELU()),
            ('sgu_%d'%i,GatingUnit(d_ff,len_sen)),
            ('fc2_%d'%i,nn.Linear(d_ff,dim)),
        ])))  for i in range(num_layers)])



    def forward(self,x):
        #embedding
        embeded=self.embedding(x)

        #ARN
        y=nn.Sequential(*self.arn)(embeded)
                
        return y

# Circle Loss

In [4]:
class CircleLoss(nn.Module):
    def __init__(self, m: float, gamma: float) -> None:
        super(CircleLoss, self).__init__()
        self.m = m
        self.gamma = gamma
        self.soft_plus = nn.Softplus()

    def forward(self, sp: Tensor, sn: Tensor, logits=None, N=None, C=None) -> Tensor:
        sp_logit = logits[sp].view(N,1)
        sn_logit = logits[sn].view(N,C-1)

        margin = self.m

        ap = torch.clamp_min(- sp_logit.detach() + 1 + margin, min=0.)
        an = torch.clamp_min(sn_logit.detach() + margin, min=0.)

        delta_p = 1 - margin
        delta_n = margin

        logit_p = - ap * (sp_logit - delta_p) * self.gamma
        logit_n = an * (sn_logit - delta_n) * self.gamma

        loss = self.soft_plus(torch.logsumexp(logit_n, dim=1) + torch.logsumexp(logit_p, dim=1))
        return loss

        

# Evaluation Function

In [5]:
def test(task,task_seen_class,task_unseen_class,reg):
    eps=0.01
    net.eval()
    
    seen_dict = {}
    seen_dict_no = {}
    seen_acc_list = []    
    
    for jj in range(len(task_seen_class)):
        mapped_class = task_seen_class[jj]
        seen_dict[mapped_class] = 0
        seen_dict_no[mapped_class] = 0
        
        
    
    unseen_dict = {}
    unseen_dict_no = {}
    unseen_acc_list = []    
        
    for jj in range(len(task_unseen_class)):
        mapped_class = task_unseen_class[jj]
        unseen_dict[mapped_class] = 0
        unseen_dict_no[mapped_class] = 0
    
    
    with torch.no_grad():
        correct=0
        Nsamples=0
        for i, (feats,targets) in enumerate(testseen_dataloader):
            feats = feats.to(DEVICE)
            targets = targets.to(DEVICE)
            logits = net(feats, attributes)
            logits[:, task_seen_class] *= reg
            pred = logits.max(1, keepdim=True)[1]
            
            for jj in range(targets.shape[0]):
                if pred[jj] == targets[jj]:
                   seen_dict[targets[jj].cpu().numpy().item()] += 1
                
                seen_dict_no[targets[jj].cpu().numpy().item()] += 1
            
            correct += pred.eq(targets.view_as(pred)).sum().item()
            Nsamples+=targets.shape[0]
            
        for jj in range(len(task_seen_class)):
            mapped_class = task_seen_class[jj]
            seen_acc_list.append(seen_dict[mapped_class]/(seen_dict_no[mapped_class] * 1.0 + 1e-5))
        
        seen_acc = np.mean(np.array(seen_acc_list))
        
        
        correct=0
        Nsamples=0
        for i, (feats,targets) in enumerate(testunseen_dataloader):
            feats = feats.to(DEVICE)
            targets = targets.to(DEVICE)
            logits = net(feats, attributes)
            logits[:, task_seen_class] *= reg
            pred = logits.max(1, keepdim=True)[1]
            for jj in range(targets.shape[0]):
                if pred[jj] == targets[jj]:
                   unseen_dict[targets[jj].cpu().numpy().item()] += 1

                unseen_dict_no[targets[jj].cpu().numpy().item()] += 1           

        for jj in range(len(task_unseen_class)):
            mapped_class = task_unseen_class[jj]
            unseen_acc_list.append(unseen_dict[mapped_class]/(unseen_dict_no[mapped_class] * 1.0 + 1e-5))

        unseen_acc = np.mean(np.array(unseen_acc_list))        
                    
        h_mean_class_wise = (2*seen_acc*unseen_acc)/(seen_acc+unseen_acc)  
        return seen_acc, unseen_acc,h_mean_class_wise 

# Initialize the Attribute Refinement Network

In [6]:
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)


arn = ARN(len_sen=1024,dim=102,d_ff=1024, num_layers=1)
arn = arn.to(DEVICE)

np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

# Class Balanced Experience Replay

In [7]:
class ER_MEM():
    def __init__(self, samples_per_class=5, total_classes=717, memory_batch_size=int(512 + 256)):
        self.trainData_memory = []
        self.trainLabels_memory = []
        self.mem_size = samples_per_class * total_classes
        self.mem_batch_size = memory_batch_size
        self.mem_sum = 0
        self.mem_counter = [0]*total_classes
        self.largest_class_list = [0]*total_classes
        self.samples_seen = [0]*total_classes
        
    def reservoir_mem_select_data(self, trainData, trainLabels):
        
        # when the memory is empty return the train data as it is
        if len(self.trainData_memory) == 0:
            return trainData, trainLabels

        # when the memory size is less than mem_batch size use all the data in memory
        elif len(self.trainData_memory) < self.mem_batch_size:
            trainData_memory_batch = torch.tensor(np.array(self.trainData_memory), dtype=torch.float32)
            trainLabels_memory_batch = torch.tensor(np.array(self.trainLabels_memory), dtype=torch.long)

            trainData = torch.cat((trainData, trainData_memory_batch), dim=0)
            trainLabels = torch.cat((trainLabels, trainLabels_memory_batch), dim=0)
                        
            return trainData, trainLabels

        # when the memory size is greater than mem_batch size randomly select data of size mem_batch
        elif len(self.trainData_memory) >= self.mem_batch_size:
            mem_index = np.random.choice(len(self.trainData_memory), self.mem_batch_size, replace=False)
            
            trainData_memory_batch = torch.tensor(np.array(self.trainData_memory)[mem_index], dtype=torch.float32)
            trainLabels_memory_batch = torch.tensor(np.array(self.trainLabels_memory)[mem_index], dtype=torch.long)

            trainData = torch.cat((trainData, trainData_memory_batch), dim=0)
            trainLabels = torch.cat((trainLabels, trainLabels_memory_batch), dim=0)

            return trainData, trainLabels
        
    
    
    def reservoir_mem_update(self, trainData_copy, trainLabels_copy):
        for ll in range(len(trainData_copy)):
            
            # when the memory is not full directly add the data to memory
            if len(self.trainData_memory) < self.mem_size:
                self.trainData_memory.append(trainData_copy[ll])
                self.trainLabels_memory.append(trainLabels_copy[ll])
                
                self.mem_counter[trainLabels_copy[ll]] = self.mem_counter[trainLabels_copy[ll]] + 1
                self.mem_sum = sum(self.mem_counter)
                
                assert self.mem_sum <= self.mem_size

            # When the memory is full update the memory based on reservoir strategy
            else:
                largest_class = np.argmax(self.mem_counter)
                self.largest_class_list[largest_class] = 1
                 
                current_class_state = self.largest_class_list[trainLabels_copy[ll]]
                
                assert largest_class < 717
                assert current_class_state <= 1
                
                if current_class_state == 1:
                    no_samples = self.mem_counter[trainLabels_copy[ll]]
                    no_samples_seen = self.samples_seen[trainLabels_copy[ll]]
                    desired_prob = no_samples/no_samples_seen
                    assert desired_prob <= 1
                    prob = np.random.uniform(0,1,1)
                    if prob <= desired_prob:
                        idx = [i for i, x in enumerate(self.trainLabels_memory) if x == trainLabels_copy[ll]]
                        assert len(idx) == self.mem_counter[trainLabels_copy[ll]]
                        idx_selected = np.random.choice(idx, 1)[0]
                        self.trainData_memory[idx_selected] = trainData_copy[ll]
                        self.trainLabels_memory[idx_selected] = trainLabels_copy[ll]                        
                        
                        
                else:
                    idx = [i for i, x in enumerate(self.trainLabels_memory) if x == largest_class]
                    assert len(idx) == self.mem_counter[largest_class]
                    idx_selected = np.random.choice(idx, 1)[0]    
                    self.trainData_memory[idx_selected] = trainData_copy[ll]
                    self.trainLabels_memory[idx_selected] = trainLabels_copy[ll]
                    
                    self.mem_counter[trainLabels_copy[ll]] += 1
                    self.mem_counter[largest_class] -= 1
                    self.mem_sum = sum(self.mem_counter)
                
                    assert self.mem_sum == self.mem_size    
                
                
            self.samples_seen[trainLabels_copy[ll]] += 1

er_mem = ER_MEM()

# Prepare the Dataset

In [8]:
print(f'<=============== Loading data for {DATASET} ===============>')
data = io.loadmat(f'{DATA_DIR}/res101.mat')
attrs_mat = io.loadmat(f'{DATA_DIR}/att_splits.mat')
feature = data['features'].T.astype(np.float32)
label = data['labels'].squeeze() - 1 # Using "-1" here and for idx to normalize to 0-index
trainval_loc = attrs_mat['trainval_loc'].squeeze() - 1
test_seen_loc = attrs_mat['test_seen_loc'].squeeze() - 1
test_unseen_loc = attrs_mat['test_unseen_loc'].squeeze() - 1
attribute = attrs_mat['att'].T

<=============== Loading data for SUN ===============>


In [9]:
x = feature[trainval_loc] # train_features
train_label = label[trainval_loc].astype(int)  # train_label
att = attribute[train_label] # train attributes

x_test = feature[test_unseen_loc]  # test_feature
test_label = label[test_unseen_loc].astype(int) # test_label
x_test_seen = feature[test_seen_loc]  #test_seen_feature
test_label_seen = label[test_seen_loc].astype(int) # test_seen_label
test_id = np.unique(test_label)   # test_id
att_pro = attribute[test_id]      # test_attribute


# train set
train_features=x
print('train_features.shape: ' + str(train_features.shape))

train_label=np.array(torch.from_numpy(train_label).unsqueeze(1))
print('train_label.shape:  '+str(train_label.shape))

# attributes
all_attributes=np.array(attribute)
print('all_attributes.shape:  '+str(all_attributes.shape))


attributes = torch.from_numpy(attribute)
attributes = attributes / attributes.norm(dim=1, keepdim=True) * np.sqrt(attributes.shape[1])

# test set
test_features=x_test
print('test_features.shape:  '+ str(test_features.shape))

test_label=np.array(torch.from_numpy(test_label).unsqueeze(1))
print('test_label.shape:  ' +str(test_label.shape))

testclasses_id = np.array(test_id)
print('testclasses_id.shape:  ' +str(testclasses_id.shape))

test_attributes = torch.from_numpy(att_pro).float()
print('test_attributes.shape:  ' +str(test_attributes.shape))


test_seen_features = torch.from_numpy(x_test_seen)
print('test_seen_features.shape:  ' +str(test_seen_features.shape))

test_seen_label = torch.from_numpy(test_label_seen)


unq_train_labels=np.unique(train_label)
unq_test_labels=np.unique(test_label)


train_features.shape: (10320, 2048)
train_label.shape:  (10320, 1)
all_attributes.shape:  (717, 102)
test_features.shape:  (1440, 2048)
test_label.shape:  (1440, 1)
testclasses_id.shape:  (72,)
test_attributes.shape:  torch.Size([72, 102])
test_seen_features.shape:  torch.Size([2580, 2048])


In [10]:
Data1=np.concatenate([train_features,test_features],0)
Label1=np.concatenate([train_label,test_label],0)

Data=np.float32(np.concatenate([Data1,x_test_seen],0))
Label=np.concatenate([Label1.squeeze(),test_label_seen],0)

In [11]:
# 20% data are kept for the test of seen classes

Nseen=Label.shape[0]
ind=random.sample(range(Nseen), int(Nseen*0.2))
seentest_data=[torch.from_numpy(Data[ind]),torch.from_numpy(Label[ind])]
    
# 80% are kep for the seen class train data
ind=np.setdiff1d(range(Nseen),ind)
seentrain_data=[torch.from_numpy(Data[ind]),torch.from_numpy(Label[ind])]

In [12]:
class all_arguments():
    cuda=True
    iterations=100
    hdim=2048
    feat_dim=2048
    attri_dim=102
    task_split=[47,47,47,48,48,48,48,48,48,48,48,48,48,48,48]
    N_class=717
    task=15
    
args=all_arguments()

class_per_task=int(args.N_class/args.task)
all_class=np.unique(Label)


# Function To Get The Data For Each Task

In [13]:
def task_data(task):
    # dataset for the seen and unseen for each task
    current_task_class=all_class[int(np.sum(args.task_split[:task-1])):int(np.sum(args.task_split[:task]))]
    task_testseen_class=all_class[:int(np.sum(args.task_split[:task]))]
    task_testunseen_class=all_class[int(np.sum(args.task_split[:task])):]
    print('current_task_class'+str(task_testseen_class))
    print('task_testseen_class: '+str(task_testseen_class.shape))
    print('task_testunseen_class: '+str(task_testunseen_class.shape))

    feature=[]
    labels=[]

    # for the seen class train data
    for i in current_task_class:
        indx=np.where(seentrain_data[1]==i)[0]
        feature.append(seentrain_data[0][indx])
        labels.append(seentrain_data[1][indx])
    task_seentrain_data=[torch.from_numpy(np.vstack(feature)),torch.from_numpy(np.hstack(labels).squeeze())]

    # for the seen class train data 

    feature=[]
    labels=[]

    # for the seen class train data
    for i in task_testseen_class:
        indx=np.where(seentest_data[1]==i)[0]
        feature.append(seentest_data[0][indx])
        labels.append(seentest_data[1][indx])
    task_seentest_data=[torch.from_numpy(np.vstack(feature)),torch.from_numpy(np.hstack(labels).squeeze())]

    feature=[]
    labels=[]

    # for the test data that are remaining from the seen class
    
    if task<args.task:
        for i in task_testunseen_class:
            indx=np.where(Label==i)[0]
            feature.append(Data[indx])
            labels.append(Label[indx])  
        task_unseentest_data=[torch.from_numpy(np.vstack(feature)),torch.from_numpy(np.hstack(labels).squeeze())]
    else:
        task_unseentest_data=[np.array([]),np.array([])]
    
    return task_seentrain_data,task_seentest_data,task_unseentest_data

# ZSL Model

In [14]:
class ClassNormalization(nn.Module):
    
    def __init__(self, feat_dim: int):
        super().__init__()
        
        self.running_mean = nn.Parameter(torch.zeros(feat_dim), requires_grad=False)
        self.running_var = nn.Parameter(torch.ones(feat_dim), requires_grad=False)
    
    def forward(self, class_feats):
        
        if self.training:
            batch_mean = class_feats.mean(dim=0)
            batch_var = class_feats.var(dim=0)
            
            # Normalizing the batch
            result = (class_feats - batch_mean.unsqueeze(0)) / (batch_var.unsqueeze(0) + 1e-5)
            
            # Updating the running mean/std
            self.running_mean.data = 0.9 * self.running_mean.data + 0.1 * batch_mean.detach()
            self.running_var.data = 0.9 * self.running_var.data + 0.1 * batch_var.detach()
        else:
            # Using accumulated statistics
            # Attention! For the test inference, we cant use batch-wise statistics,
            # only the accumulated ones. Otherwise, it will be quite transductive
            result = (class_feats - self.running_mean.unsqueeze(0)) / (self.running_var.unsqueeze(0) + 1e-5)
        
        return result

In [15]:
class ZSLModel(nn.Module):
    def __init__(self, attr_dim: int, hid_dim: int, proto_dim: int):
        super().__init__()
        
        # Attribute to Feature Embedding
        self.model = nn.Sequential(
            nn.Linear(attr_dim, hid_dim),
            nn.ReLU(),
            
            nn.Linear(hid_dim, hid_dim),
            ClassNormalization(hid_dim),
            nn.ReLU(),
            
            ClassNormalization(hid_dim),
            nn.Linear(hid_dim, proto_dim),
            nn.ReLU(),
        )
        
        # Weight Initialization
        weight_var = 1 / (hid_dim * proto_dim)
        b = np.sqrt(3 * weight_var)
        self.model[-2].weight.data.uniform_(-b, b)
        
    def forward(self, x, attrs, train=False):
        attrs = arn(attrs)
        protos = self.model(attrs)
        
        # Feature-Prototype Combiner 
        x_ns = 5 * x / x.norm(dim=1, keepdim=True)
        protos_ns = 5 * protos / protos.norm(dim=1, keepdim=True)
        logits = x_ns @ protos_ns.t()
        
        x_nss = 1.0 * x / x.norm(dim=1, keepdim=True)
        protos_nss = 1.0 * protos / protos.norm(dim=1, keepdim=True)
        logits_nn = x_nss @ protos_nss.t()
        
        if train:
            return logits, protos, logits_nn
        else:
            return logits

# Function for Training The Model Task-Wise

In [16]:
loss_circle1 = CircleLoss(m=0.4, gamma=0.5)

def do_learning(net, optimizer, task, epoch):

    net.train()
    
    for (feats_s, targets_s) in train_dataloader:
        attrs=attributes[task_seen_class]
        attrs_s=attributes[task_seen_class]
        
                
        if task > 1:
            feats, targets = er_mem.reservoir_mem_select_data(torch.tensor(feats_s), torch.tensor(targets_s))
        else:
            feats, targets = feats_s, targets_s
        
        feats, targets = feats.to(DEVICE), targets.to(DEVICE) 
        
        N, D = feats.shape
        C, K = attrs_s.cpu().numpy().shape
                
               
        logits, proto_s, logits_nn = net(feats, attrs, True)
        
        targets_one_hot = F.one_hot(targets, C)
        
        sp_bool = targets_one_hot[:] == 1
        sn_bool = targets_one_hot[:] != 1
        
        loss_circle = torch.mean(loss_circle1(sp_bool, sn_bool, logits=logits_nn, N=N, C=C))
        
        loss = F.cross_entropy(logits, targets) + loss_circle * 1.2
        
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch == args.iterations - 1:
            with torch.no_grad():
                er_mem.reservoir_mem_update(feats_s.numpy(), targets_s.numpy())

    return loss.item()


# Begin Training

In [17]:
attributes=attributes.to(torch.float32).to(DEVICE)

Overall_hmean=[]

print('\n<=============== Training Samples %s===============>')

task_best_hmean=[]
for task in range(args.task-1):
    best_hmean=0
    reg=1.0

    print('\n<=============== Training task %s===============>' % task)
    # Build model, optimizer, and set states
    if task == 0:
        net = ZSLModel(args.attri_dim, args.hdim, args.feat_dim).to(DEVICE)
        if args.cuda:
            net.cuda()

    optimizer = torch.optim.Adam(net.parameters(), lr=0.0005, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=25)


    train_seen,test_seen,test_unseen=task_data(task+1)
    imgs,labels=train_seen[0],train_seen[1]

    task_seen_class=all_class[:int(np.sum(args.task_split[:task+1]))]
    task_unseen_class=all_class[int(np.sum(args.task_split[:task+1])):]

    train_dataloader = DataLoader(TensorDataset(imgs,labels), batch_size=256, shuffle=True) # 256
    testseen_dataloader = DataLoader(TensorDataset(test_seen[0],test_seen[1]), batch_size=2048)
    testunseen_dataloader = DataLoader(TensorDataset(test_unseen[0],test_unseen[1]), batch_size=2048)

    for epoch in range(args.iterations):          

        loss = do_learning(net, optimizer, task+1, epoch)
        ACC_seen, ACC_unseen,H_mean=test(task+1,task_seen_class,task_unseen_class,reg)

        if H_mean>best_hmean:
            best_hmean=H_mean
            result=[ACC_seen, ACC_unseen,H_mean]
        if epoch%1==0:
            print('[ Epoch: %d  TLoss: %0.3f    Best_HMean:  %0.3f ACC_seen: %0.3f, ACC_unseen: %0.3f'
                  %(epoch,loss,best_hmean,ACC_seen,ACC_unseen))

        scheduler.step()

    task_best_hmean.append(result)

    print(task_best_hmean)
Overall_hmean.append(task_best_hmean)
print(Overall_hmean)



<=============== Training Samples %s===============>

<=============== Training task 0===============>
current_task_class[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
task_testseen_class: (47,)
task_testunseen_class: (670,)
[ Epoch: 0  TLoss: 6.510    Best_HMean:  0.039 ACC_seen: 0.336, ACC_unseen: 0.021
[ Epoch: 1  TLoss: 5.860    Best_HMean:  0.057 ACC_seen: 0.388, ACC_unseen: 0.031
[ Epoch: 2  TLoss: 5.620    Best_HMean:  0.072 ACC_seen: 0.460, ACC_unseen: 0.039
[ Epoch: 3  TLoss: 5.357    Best_HMean:  0.076 ACC_seen: 0.445, ACC_unseen: 0.042
[ Epoch: 4  TLoss: 5.287    Best_HMean:  0.087 ACC_seen: 0.476, ACC_unseen: 0.048
[ Epoch: 5  TLoss: 5.217    Best_HMean:  0.104 ACC_seen: 0.475, ACC_unseen: 0.058
[ Epoch: 6  TLoss: 5.117    Best_HMean:  0.118 ACC_seen: 0.450, ACC_unseen: 0.068
[ Epoch: 7  TLoss: 5.002    Best_HMean:  0.126 ACC_seen: 0.362, ACC_unseen: 0.077
[ Epoch: 8  TLoss: 5.

[ Epoch: 95  TLoss: 4.703    Best_HMean:  0.126 ACC_seen: 0.092, ACC_unseen: 0.077
[ Epoch: 96  TLoss: 4.705    Best_HMean:  0.126 ACC_seen: 0.092, ACC_unseen: 0.077
[ Epoch: 97  TLoss: 4.686    Best_HMean:  0.126 ACC_seen: 0.092, ACC_unseen: 0.077
[ Epoch: 98  TLoss: 4.705    Best_HMean:  0.126 ACC_seen: 0.092, ACC_unseen: 0.077
[ Epoch: 99  TLoss: 4.705    Best_HMean:  0.126 ACC_seen: 0.092, ACC_unseen: 0.077
[[0.362410360814611, 0.07656712589554154, 0.12642434093966576]]

<=============== Training task 1===============>
current_task_class[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93]
task_testseen_class: (94,)
task_testunseen_class: (623,)
[ Epoch: 0  TLoss: 6.306    Best_HMean:  0.151 ACC_seen: 0.266, ACC_unseen: 0.106
[ Epoch: 1  TLoss: 6.043   

[ Epoch: 89  TLoss: 5.615    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 90  TLoss: 5.607    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 91  TLoss: 5.615    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 92  TLoss: 5.617    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 93  TLoss: 5.617    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 94  TLoss: 5.612    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 95  TLoss: 5.617    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 96  TLoss: 5.615    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 97  TLoss: 5.608    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 98  TLoss: 5.618    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[ Epoch: 99  TLoss: 5.667    Best_HMean:  0.164 ACC_seen: 0.156, ACC_unseen: 0.122
[[0.362410360814611, 0.07656712589554154, 0.12642434093966576], [0.2793431795735104, 0.

[ Epoch: 78  TLoss: 6.198    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 79  TLoss: 6.201    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 80  TLoss: 6.203    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 81  TLoss: 6.195    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 82  TLoss: 6.194    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 83  TLoss: 6.196    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 84  TLoss: 6.200    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 85  TLoss: 6.191    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 86  TLoss: 6.195    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 87  TLoss: 6.196    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 88  TLoss: 6.195    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Epoch: 89  TLoss: 6.184    Best_HMean:  0.189 ACC_seen: 0.183, ACC_unseen: 0.156
[ Ep

[ Epoch: 64  TLoss: 6.614    Best_HMean:  0.217 ACC_seen: 0.210, ACC_unseen: 0.187
[ Epoch: 65  TLoss: 6.600    Best_HMean:  0.217 ACC_seen: 0.210, ACC_unseen: 0.187
[ Epoch: 66  TLoss: 6.613    Best_HMean:  0.217 ACC_seen: 0.210, ACC_unseen: 0.187
[ Epoch: 67  TLoss: 6.596    Best_HMean:  0.217 ACC_seen: 0.210, ACC_unseen: 0.187
[ Epoch: 68  TLoss: 6.613    Best_HMean:  0.217 ACC_seen: 0.210, ACC_unseen: 0.187
[ Epoch: 69  TLoss: 6.589    Best_HMean:  0.217 ACC_seen: 0.210, ACC_unseen: 0.188
[ Epoch: 70  TLoss: 6.591    Best_HMean:  0.217 ACC_seen: 0.211, ACC_unseen: 0.187
[ Epoch: 71  TLoss: 6.608    Best_HMean:  0.217 ACC_seen: 0.209, ACC_unseen: 0.187
[ Epoch: 72  TLoss: 6.621    Best_HMean:  0.217 ACC_seen: 0.209, ACC_unseen: 0.187
[ Epoch: 73  TLoss: 6.603    Best_HMean:  0.217 ACC_seen: 0.209, ACC_unseen: 0.187
[ Epoch: 74  TLoss: 6.605    Best_HMean:  0.217 ACC_seen: 0.208, ACC_unseen: 0.187
[ Epoch: 75  TLoss: 6.601    Best_HMean:  0.217 ACC_seen: 0.208, ACC_unseen: 0.187
[ Ep

...

[ Epoch: 2  TLoss: 8.091    Best_HMean:  0.311 ACC_seen: 0.317, ACC_unseen: 0.305
[ Epoch: 3  TLoss: 8.047    Best_HMean:  0.311 ACC_seen: 0.315, ACC_unseen: 0.301
[ Epoch: 4  TLoss: 8.047    Best_HMean:  0.311 ACC_seen: 0.315, ACC_unseen: 0.300
[ Epoch: 5  TLoss: 8.026    Best_HMean:  0.311 ACC_seen: 0.310, ACC_unseen: 0.302
[ Epoch: 6  TLoss: 8.006    Best_HMean:  0.311 ACC_seen: 0.315, ACC_unseen: 0.307
[ Epoch: 7  TLoss: 8.024    Best_HMean:  0.314 ACC_seen: 0.321, ACC_unseen: 0.307
[ Epoch: 8  TLoss: 7.968    Best_HMean:  0.314 ACC_seen: 0.322, ACC_unseen: 0.307
[ Epoch: 9  TLoss: 7.978    Best_HMean:  0.314 ACC_seen: 0.317, ACC_unseen: 0.306
[ Epoch: 10  TLoss: 7.993    Best_HMean:  0.314 ACC_seen: 0.318, ACC_unseen: 0.306
[ Epoch: 11  TLoss: 7.981    Best_HMean:  0.314 ACC_seen: 0.311, ACC_unseen: 0.314
[ Epoch: 12  TLoss: 7.995    Best_HMean:  0.314 ACC_seen: 0.309, ACC_unseen: 0.315
[ Epoch: 13  TLoss: 7.971    Best_HMean:  0.314 ACC_seen: 0.304, ACC_unseen: 0.315
[ Epoch: 14 

[ Epoch: 0  TLoss: 8.484    Best_HMean:  0.307 ACC_seen: 0.311, ACC_unseen: 0.303
[ Epoch: 1  TLoss: 8.374    Best_HMean:  0.311 ACC_seen: 0.324, ACC_unseen: 0.299
[ Epoch: 2  TLoss: 8.306    Best_HMean:  0.317 ACC_seen: 0.327, ACC_unseen: 0.308
[ Epoch: 3  TLoss: 8.279    Best_HMean:  0.317 ACC_seen: 0.320, ACC_unseen: 0.301
[ Epoch: 4  TLoss: 8.236    Best_HMean:  0.317 ACC_seen: 0.317, ACC_unseen: 0.299
[ Epoch: 5  TLoss: 8.202    Best_HMean:  0.317 ACC_seen: 0.323, ACC_unseen: 0.298
[ Epoch: 6  TLoss: 8.193    Best_HMean:  0.317 ACC_seen: 0.322, ACC_unseen: 0.301
[ Epoch: 7  TLoss: 8.172    Best_HMean:  0.317 ACC_seen: 0.324, ACC_unseen: 0.294
[ Epoch: 8  TLoss: 8.155    Best_HMean:  0.317 ACC_seen: 0.320, ACC_unseen: 0.296
[ Epoch: 9  TLoss: 8.155    Best_HMean:  0.317 ACC_seen: 0.319, ACC_unseen: 0.300
[ Epoch: 10  TLoss: 8.129    Best_HMean:  0.317 ACC_seen: 0.313, ACC_unseen: 0.298
[ Epoch: 11  TLoss: 8.132    Best_HMean:  0.317 ACC_seen: 0.311, ACC_unseen: 0.300
[ Epoch: 12  T

[ Epoch: 99  TLoss: 8.044    Best_HMean:  0.317 ACC_seen: 0.301, ACC_unseen: 0.322
[[0.362410360814611, 0.07656712589554154, 0.12642434093966576], [0.2793431795735104, 0.11597104955861648, 0.16389858666763735], [0.24825155217549252, 0.15225686831601026, 0.18875010838695233], [0.26659650816827546, 0.1833332416667125, 0.2172605926922656], [0.29156026543504593, 0.2041665645833844, 0.24015991936801118], [0.29192475565473125, 0.22175914837968508, 0.2520498878649059], [0.3149413446758019, 0.23463529934901697, 0.2689210230777258], [0.32659144220220343, 0.2630951065476848, 0.2914246915277531], [0.30462223654886766, 0.27187486406256794, 0.28731845854669674], [0.3347007995023005, 0.28312485843757074, 0.306760055236337], [0.3083210300133327, 0.32265608867195567, 0.3153257214990356], [0.3271929207856598, 0.30833317916674374, 0.31748321107268634]]

<=============== Training task 12===============>
current_task_class[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  2

[ Epoch: 57  TLoss: 8.062    Best_HMean:  0.321 ACC_seen: 0.318, ACC_unseen: 0.319
[ Epoch: 58  TLoss: 8.056    Best_HMean:  0.321 ACC_seen: 0.319, ACC_unseen: 0.319
[ Epoch: 59  TLoss: 8.051    Best_HMean:  0.321 ACC_seen: 0.319, ACC_unseen: 0.319
[ Epoch: 60  TLoss: 8.049    Best_HMean:  0.321 ACC_seen: 0.319, ACC_unseen: 0.319
[ Epoch: 61  TLoss: 8.064    Best_HMean:  0.321 ACC_seen: 0.319, ACC_unseen: 0.318
[ Epoch: 62  TLoss: 8.035    Best_HMean:  0.321 ACC_seen: 0.319, ACC_unseen: 0.318
[ Epoch: 63  TLoss: 8.047    Best_HMean:  0.321 ACC_seen: 0.318, ACC_unseen: 0.319
[ Epoch: 64  TLoss: 8.048    Best_HMean:  0.321 ACC_seen: 0.319, ACC_unseen: 0.319
[ Epoch: 65  TLoss: 8.049    Best_HMean:  0.321 ACC_seen: 0.319, ACC_unseen: 0.320
[ Epoch: 66  TLoss: 8.061    Best_HMean:  0.321 ACC_seen: 0.319, ACC_unseen: 0.320
[ Epoch: 67  TLoss: 8.054    Best_HMean:  0.321 ACC_seen: 0.319, ACC_unseen: 0.320
[ Epoch: 68  TLoss: 8.042    Best_HMean:  0.321 ACC_seen: 0.318, ACC_unseen: 0.321
[ Ep

[ Epoch: 12  TLoss: 8.213    Best_HMean:  0.323 ACC_seen: 0.336, ACC_unseen: 0.303
[ Epoch: 13  TLoss: 8.225    Best_HMean:  0.325 ACC_seen: 0.336, ACC_unseen: 0.315
[ Epoch: 14  TLoss: 8.222    Best_HMean:  0.331 ACC_seen: 0.339, ACC_unseen: 0.324
[ Epoch: 15  TLoss: 8.197    Best_HMean:  0.331 ACC_seen: 0.333, ACC_unseen: 0.327
[ Epoch: 16  TLoss: 8.206    Best_HMean:  0.336 ACC_seen: 0.333, ACC_unseen: 0.339
[ Epoch: 17  TLoss: 8.190    Best_HMean:  0.340 ACC_seen: 0.342, ACC_unseen: 0.339
[ Epoch: 18  TLoss: 8.195    Best_HMean:  0.340 ACC_seen: 0.338, ACC_unseen: 0.324
[ Epoch: 19  TLoss: 8.221    Best_HMean:  0.340 ACC_seen: 0.337, ACC_unseen: 0.326
[ Epoch: 20  TLoss: 8.206    Best_HMean:  0.340 ACC_seen: 0.336, ACC_unseen: 0.321
[ Epoch: 21  TLoss: 8.191    Best_HMean:  0.340 ACC_seen: 0.335, ACC_unseen: 0.311
[ Epoch: 22  TLoss: 8.222    Best_HMean:  0.340 ACC_seen: 0.328, ACC_unseen: 0.310
[ Epoch: 23  TLoss: 8.197    Best_HMean:  0.340 ACC_seen: 0.328, ACC_unseen: 0.311
[ Ep

In [18]:
print(np.mean(Overall_hmean[0],0))

[0.30856039 0.23551058 0.25979628]
